In [3]:
from web3 import Web3
import pandas as pd
import numpy as np
import time
import os
from datetime import datetime, timedelta

# ============================================================================= 
# WEEK 6: CORRECTED DATA RETRIEVAL AND PROCESSING FOR TABLEAU/CHARTS
# =============================================================================

print("📥 Starting Week 6: Data Retrieval and Processing (CORRECTED VERSION)")
print("🎯 Output: Clean dataset ready for Tableau dashboards and line charts")
print("=" * 70)

# Step 1: Connect to Ganache blockchain
print("🔗 Connecting to blockchain...")

ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Make sure Ganache is running.")
    exit()

# Contract details
contract_address = web3.to_checksum_address("0x327ef98ba37138026c79783e02d2a752ecd58f9e")
contract_abi = [
    {
        "inputs": [{"internalType": "address","name": "device","type": "address"}],
        "name": "authorizeDevice","outputs": [],"stateMutability": "nonpayable","type": "function"
    },
    {
        "inputs": [{"internalType": "address","name": "device","type": "address"}],
        "name": "revokeDevice","outputs": [],"stateMutability": "nonpayable","type": "function"
    },
    {
        "inputs": [
            {"internalType": "string","name": "deviceID","type": "string"},
            {"internalType": "string","name": "dataType","type": "string"},
            {"internalType": "string","name": "value","type": "string"}
        ],
        "name": "storeData","outputs": [],"stateMutability": "nonpayable","type": "function"
    },
    {
        "inputs": [],"stateMutability": "nonpayable","type": "constructor"
    },
    {
        "inputs": [{"internalType": "address","name": "","type": "address"}],
        "name": "authorizedDevices","outputs": [{"internalType": "bool","name": "","type": "bool"}],
        "stateMutability": "view","type": "function"
    },
    {
        "inputs": [
            {"internalType": "string","name": "deviceID","type": "string"},
            {"internalType": "uint256","name": "index","type": "uint256"}
        ],
        "name": "getDataByIndex",
        "outputs": [
            {"internalType": "string","name": "","type": "string"},
            {"internalType": "string","name": "","type": "string"},
            {"internalType": "uint256","name": "","type": "uint256"}
        ],
        "stateMutability": "view","type": "function"
    },
    {
        "inputs": [{"internalType": "string","name": "deviceID","type": "string"}],
        "name": "getDataCount",
        "outputs": [{"internalType": "uint256","name": "","type": "uint256"}],
        "stateMutability": "view","type": "function"
    },
    {
        "inputs": [],"name": "owner",
        "outputs": [{"internalType": "address","name": "","type": "address"}],
        "stateMutability": "view","type": "function"
    }
]

# Load the smart contract
contract = web3.eth.contract(address=contract_address, abi=contract_abi)
web3.eth.default_account = web3.eth.accounts[0]
print("✅ Connected to Smart Contract")

# Step 2: Find and load CSV file for reference data
print("\n📁 Looking for logistics data file...")

possible_files = [
    "LogisticsData.csv",
    "Logistics-Data.csv", 
    "logistics_data.csv",
    "logistics-data.csv",
    "IoT_Data.csv",
    "iot_data.csv"
]

df_original = None
csv_filename = None

# Check current directory for CSV files
current_files = [f for f in os.listdir('.') if f.endswith('.csv')]
print(f"📋 CSV files found: {current_files}")

# Try to find the logistics data file
for filename in possible_files:
    if os.path.exists(filename):
        try:
            df_original = pd.read_csv(filename)
            csv_filename = filename
            print(f"✅ Found and loaded: {filename}")
            print(f"📊 Data shape: {df_original.shape}")
            break
        except Exception as e:
            print(f"❌ Error loading {filename}: {e}")

# If still no file found, try the first CSV file in directory
if df_original is None and current_files:
    try:
        csv_filename = current_files[0]
        df_original = pd.read_csv(csv_filename)
        print(f"✅ Using first available CSV: {csv_filename}")
    except Exception as e:
        print(f"❌ Error loading {csv_filename}: {e}")

if df_original is None:
    print("❌ No logistics data file found. Please ensure you have the logistics CSV file.")
    exit()

# Verify required columns exist
required_columns = ['package_id', 'temperature', 'humidity', 'shock']
missing_columns = [col for col in required_columns if col not in df_original.columns]

if missing_columns:
    print(f"❌ Missing required columns: {missing_columns}")
    print("📋 Available columns:", list(df_original.columns))
    exit()

print("✅ All required columns found!")

# Step 3: Get blockchain data count
print("\n📊 Checking blockchain data...")

device_ids = df_original["package_id"].unique()
total_records = 0

for device_id in device_ids:
    try:
        device_count = contract.functions.getDataCount(str(device_id)).call()
        total_records += device_count
    except Exception as e:
        print(f"⚠️ Error checking records for {device_id}: {e}")

print(f"📦 Total packages: {len(device_ids)}")
print(f"📈 Blockchain records found: {total_records}")

if total_records == 0:
    print("⚠️ No data on blockchain!")
    print("🚀 Will automatically store CSV data to blockchain first...")
    
    # STORE ALL DATA TO BLOCKCHAIN
    print(f"\n📤 Storing IoT data to blockchain...")
    
    # Authorize the current account first
    try:
        print("🔑 Authorizing device...")
        auth_txn = contract.functions.authorizeDevice(web3.eth.default_account).transact({
            'from': web3.eth.default_account,
            'gas': 3000000
        })
        web3.eth.wait_for_transaction_receipt(auth_txn)
        print("✅ Device authorized successfully!")
    except Exception as e:
        print(f"⚠️ Authorization issue (may already exist): {e}")
    
    # Store ALL data from CSV to blockchain
    stored_count = 0
    failed_count = 0
    expected_records = len(df_original) * 3  # 3 sensors per package
    
    print(f"📦 Storing data from {len(df_original)} packages...")
    print(f"🎯 Will create {expected_records} blockchain records...")
    
    for index, row in df_original.iterrows():
        device_id = str(row["package_id"])
        
        # Store temperature, humidity, and shock data with validation
        sensors = [
            ("temperature", abs(float(row["temperature"]))),  # Ensure positive
            ("humidity", abs(float(row["humidity"]))),        # Ensure positive
            ("shock", abs(float(row["shock"])))               # Ensure positive
        ]
        
        # Show progress every 10 packages
        if index % 10 == 0:
            print(f"📊 Progress: {index+1}/{len(df_original)} packages ({stored_count} records stored)...")
        
        for sensor_type, sensor_value in sensors:
            # Additional validation: Ensure reasonable ranges
            if sensor_type == "temperature" and sensor_value > 60:
                sensor_value = 60  # Cap extreme temperatures
            elif sensor_type == "humidity" and sensor_value > 100:
                sensor_value = 100  # Cap humidity at 100%
            elif sensor_type == "shock" and sensor_value > 10:
                sensor_value = 10  # Cap extreme shock values
            
            try:
                txn = contract.functions.storeData(device_id, sensor_type, str(sensor_value)).transact({
                    'from': web3.eth.default_account,
                    'gas': 3000000
                })
                web3.eth.wait_for_transaction_receipt(txn)
                stored_count += 1
                
                # Show detailed progress for first few records
                if stored_count <= 5:
                    print(f"   🛰️ Stored [{device_id}] - {sensor_type}: {sensor_value}")
                
                # Small delay to prevent overwhelming Ganache
                time.sleep(0.05)
                
            except Exception as e:
                failed_count += 1
                print(f"   ❌ Failed to store {sensor_type} for {device_id}: {e}")
    
    print(f"\n🎉 Data storage completed!")
    print(f"✅ Successfully stored: {stored_count} records")
    print(f"❌ Failed to store: {failed_count} records")
    if stored_count + failed_count > 0:
        print(f"📊 Success rate: {(stored_count/(stored_count+failed_count)*100):.1f}%")
    
    # Update total_records for retrieval
    total_records = stored_count
    print(f"📈 Blockchain now contains {total_records} records")
    
    if total_records == 0:
        print("❌ No data was stored successfully. Check blockchain connection.")
        exit()

# NOW RETRIEVE THE DATA FROM BLOCKCHAIN
print(f"\n📥 Retrieving {total_records} records from blockchain...")
print("🕐 Using ORIGINAL CSV timestamps (not blockchain timestamps)")
print("📅 Timeline will match your original logistics data")

data = []
retrieved_count = 0
timestamp_errors = 0

# Show expected timestamp range from CSV
csv_timestamps = pd.to_datetime(df_original["timestamp"])
print(f"📊 Original data timeline: {csv_timestamps.min()} to {csv_timestamps.max()}")

# Test first device to debug
if len(device_ids) > 0:
    test_device = device_ids[0]
    print(f"🔍 Testing first device: {test_device}")
    try:
        test_count = contract.functions.getDataCount(str(test_device)).call()
        print(f"📊 Test device has {test_count} records")
        
        if test_count > 0:
            test_record = contract.functions.getDataByIndex(str(test_device), 0).call()
            print(f"🧪 Test record structure: {test_record}")
            print(f"   Data Type: {test_record[0]}")
            print(f"   Value: {test_record[1]}")
            print(f"   Blockchain Timestamp: {test_record[2]} (will ignore this)")
            
            # Show what CSV timestamp we'll use instead
            test_csv_data = df_original[df_original["package_id"] == test_device].iloc[0]
            print(f"   CSV Timestamp: {test_csv_data['timestamp']} (will use this)")
    except Exception as e:
        print(f"❌ Test failed: {e}")

for device_id in device_ids:
    try:
        device_count = contract.functions.getDataCount(str(device_id)).call()
        
        if device_count > 0:
            # Get original data for location/status info
            device_original_data = df_original[df_original["package_id"] == device_id].iloc[0]
            
            # Get each record for this device from blockchain
            for i in range(device_count):
                try:
                    # Get record from blockchain: [dataType, value, timestamp]
                    blockchain_record = contract.functions.getDataByIndex(str(device_id), i).call()
                    
                    # DEBUG: Print first few records
                    if retrieved_count < 3:
                        print(f"🔍 Record {retrieved_count}: {blockchain_record}")
                        print(f"   Using CSV timestamp: {device_original_data['timestamp']}")
                    
                    # CORRECTED: Use ORIGINAL CSV timestamp, not blockchain timestamp
                    # Get the original timestamp from CSV data for this device
                    original_timestamp = device_original_data["timestamp"]
                    
                    # Convert to proper datetime format
                    try:
                        timestamp = pd.to_datetime(original_timestamp)
                    except Exception as e:
                        # Fallback: use a reasonable timestamp
                        base_time = datetime.now() - timedelta(days=7)
                        timestamp = base_time + timedelta(minutes=retrieved_count)
                        timestamp_errors += 1
                        print(f"⚠️ Timestamp conversion error for {device_id}: {e}")
                    
                    # Structure data for Tableau/Charts
                    record_data = {
                        "timestamp": timestamp,  # ✅ Using ORIGINAL CSV timestamp
                        "device_id": str(device_id),
                        "data_type": blockchain_record[0],  # temperature/humidity/shock
                        "data_value": blockchain_record[1],  # raw value as string
                        "location": device_original_data.get("location", "Unknown"),
                        "closest_city": device_original_data.get("closest_city", "Unknown"),
                        "status": device_original_data.get("status", "In Transit"),
                        "origin": device_original_data.get("origin", "Unknown"),
                        "destination": device_original_data.get("destination", "Unknown")
                    }
                    
                    # DEBUG: Print first record structure
                    if retrieved_count == 0:
                        print(f"🧪 First record structure: {record_data}")
                    
                    data.append(record_data)
                    retrieved_count += 1
                    
                    # Show progress
                    if retrieved_count % 100 == 0:
                        print(f"📥 Retrieved {retrieved_count}/{total_records} records...")
                        
                except Exception as e:
                    print(f"❌ Error retrieving record {i} for {device_id}: {e}")
                    
    except Exception as e:
        print(f"❌ Error processing device {device_id}: {e}")

print(f"✅ Retrieved {len(data)} records from blockchain!")
print(f"📊 Data list length: {len(data)}")
if len(data) > 0:
    print(f"🔍 Sample data item keys: {list(data[0].keys())}")
    print(f"🕐 Using original CSV timeline: {data[0]['timestamp']}")
if timestamp_errors > 0:
    print(f"⚠️ Fixed {timestamp_errors} timestamp conversion errors")

# Step 4: PROCESS DATA FOR VISUALIZATION
print(f"\n🔧 Processing data for Tableau and line charts...")

# Convert to DataFrame
df = pd.DataFrame(data)

# DEBUG: Check what we actually got
print(f"📊 DataFrame shape: {df.shape}")
if len(df) > 0:
    print(f"📋 Columns found: {list(df.columns)}")
    print(f"📝 First record sample:")
    print(df.head(1).to_dict('records'))
else:
    print("❌ No data retrieved! Check blockchain connection.")
    exit()

# Check if DataFrame has the expected columns
expected_columns = ['timestamp', 'device_id', 'data_type', 'data_value']
missing_columns = [col for col in expected_columns if col not in df.columns]

if missing_columns:
    print(f"❌ Missing columns in DataFrame: {missing_columns}")
    print(f"📋 Available columns: {list(df.columns)}")
    print("🔍 Sample data structure:")
    if len(data) > 0:
        print(data[0])
    exit()

# Ensure proper timestamp format
print("🕐 Converting timestamps...")
try:
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    print("✅ Timestamps converted successfully")
except Exception as e:
    print(f"❌ Timestamp conversion failed: {e}")
    print("🔍 Sample timestamp values:")
    print(df["timestamp"].head())
    exit()

# Extract numeric values with better parsing
print("🔢 Extracting numeric values...")
print("🛡️ Enforcing positive values only (IoT sensors don't produce negative readings)")

def extract_numeric(value_str):
    """Extract numeric value from string, handling various formats - NO NEGATIVE VALUES
    
    IoT sensors for temperature, humidity, and shock should not produce negative readings:
    - Temperature: Absolute zero is 0°C minimum in normal conditions
    - Humidity: Cannot be below 0% 
    - Shock: Measured as magnitude (always positive)
    """
    try:
        # Remove common units and convert to float
        clean_value = str(value_str).replace('°C', '').replace('%', '').replace('g', '').strip()
        # Extract first number found - EXCLUDE negative numbers
        import re
        numbers = re.findall(r'\d+\.?\d*', clean_value)  # Removed '-?' to prevent negative
        if numbers:
            value = float(numbers[0])
            # Ensure no negative values - convert to absolute value
            return abs(value)
        return 0.0
    except:
        return 0.0

df["numeric_value"] = df["data_value"].apply(extract_numeric)

# Additional validation: Ensure NO negative values anywhere
negative_count = (df["numeric_value"] < 0).sum()
if negative_count > 0:
    print(f"⚠️ Found {negative_count} negative values, converting to positive...")
    df["numeric_value"] = df["numeric_value"].abs()  # Use assignment instead of inplace
    print(f"✅ All negative values converted to positive")

# Data validation and cleaning
print("🧹 Cleaning and validating data...")

# Remove any completely invalid records
initial_count = len(df)
df = df[df["data_type"].isin(["temperature", "humidity", "shock"])]
df = df[df["numeric_value"].notna()]

# STRICT VALIDATION: No negative values allowed
negative_values = df["numeric_value"] < 0
if negative_values.sum() > 0:
    print(f"⚠️ Found {negative_values.sum()} negative values, converting to absolute values...")
    df.loc[negative_values, "numeric_value"] = df.loc[negative_values, "numeric_value"].abs()

# Additional sensor-specific validation
print("🔍 Applying sensor-specific validation...")

# Temperature validation (reasonable range: 0-60°C)
temp_data = df["data_type"] == "temperature"
extreme_temp = (df["numeric_value"] > 60) & temp_data
if extreme_temp.sum() > 0:
    print(f"⚠️ Found {extreme_temp.sum()} extreme temperature values (>60°C), capping at 60°C")
    df.loc[extreme_temp, "numeric_value"] = 60

# Humidity validation (reasonable range: 0-100%)
humidity_data = df["data_type"] == "humidity"
extreme_humidity = (df["numeric_value"] > 100) & humidity_data
if extreme_humidity.sum() > 0:
    print(f"⚠️ Found {extreme_humidity.sum()} extreme humidity values (>100%), capping at 100%")
    df.loc[extreme_humidity, "numeric_value"] = 100

# Shock validation (reasonable range: 0-10g)
shock_data = df["data_type"] == "shock"
extreme_shock = (df["numeric_value"] > 10) & shock_data
if extreme_shock.sum() > 0:
    print(f"⚠️ Found {extreme_shock.sum()} extreme shock values (>10g), capping at 10g")
    df.loc[extreme_shock, "numeric_value"] = 10

# Final validation: Ensure ALL values are positive
final_negative_check = (df["numeric_value"] < 0).sum()
if final_negative_check > 0:
    print(f"❌ Still found {final_negative_check} negative values after cleaning!")
    df["numeric_value"] = df["numeric_value"].abs()  # Use assignment instead of inplace
    print("✅ Forced all values to positive")

final_count = len(df)

if final_count < initial_count:
    print(f"⚠️ Removed {initial_count - final_count} invalid records")

# Handle any remaining missing values with positive defaults
df = df.fillna({"numeric_value": 0})  # Fix for pandas FutureWarning
df = df.fillna("Unknown")  # Fill other missing values

# Final check: Confirm no negative values exist
min_value = df["numeric_value"].min()
if min_value < 0:
    print(f"❌ ERROR: Still have negative values! Minimum: {min_value}")
    exit()
else:
    print(f"✅ All numeric values are positive. Minimum value: {min_value:.2f}")

# Add record ID for Tableau (helps with aggregation issues)
df["record_id"] = range(1, len(df) + 1)

# Sort by timestamp for proper line chart display
df = df.sort_values(["timestamp", "device_id", "data_type"]).reset_index(drop=True)

# Step 5: DATA QUALITY VALIDATION
print(f"\n📊 Data Quality Report:")
print(f"   ✅ Total records: {len(df)}")
print(f"   📦 Unique packages: {df['device_id'].nunique()}")
print(f"   📈 Sensor types: {', '.join(sorted(df['data_type'].unique()))}")
print(f"   🌍 Cities: {df['closest_city'].nunique()}")
print(f"   📅 Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")

# Sensor data validation
print(f"\n🔍 Sensor Data Validation (ALL VALUES POSITIVE):")
sensor_stats = {}

for sensor_type in ["temperature", "humidity", "shock"]:
    sensor_data = df[df['data_type'] == sensor_type]['numeric_value']
    if len(sensor_data) > 0:
        stats = {
            'count': len(sensor_data),
            'min': sensor_data.min(),
            'max': sensor_data.max(), 
            'mean': sensor_data.mean(),
            'std': sensor_data.std()
        }
        sensor_stats[sensor_type] = stats
        print(f"   {sensor_type}: {stats['count']} readings")
        print(f"     Range: {stats['min']:.2f} - {stats['max']:.2f}")
        print(f"     Average: {stats['mean']:.2f} ± {stats['std']:.2f}")
        
        # Validation: Ensure no negative values
        if stats['min'] < 0:
            print(f"     ❌ ERROR: Found negative values in {sensor_type}!")
        else:
            print(f"     ✅ All {sensor_type} values are positive")

# Data distribution check
print(f"\n📊 Records per sensor type:")
for sensor in sorted(df['data_type'].unique()):
    count = len(df[df['data_type'] == sensor])
    percentage = (count / len(df)) * 100
    print(f"   {sensor}: {count} records ({percentage:.1f}%)")

# Timeline validation
timeline_span = df['timestamp'].max() - df['timestamp'].min()
print(f"\n⏰ Timeline validation (using ORIGINAL CSV timeline):")
print(f"   Time span: {timeline_span}")
print(f"   Records per day: {len(df) / max(1, timeline_span.days):.1f}")
print(f"   📅 Data covers: {df['timestamp'].min()} to {df['timestamp'].max()}")

# Step 6: SAVE CLEAN DATASET FOR TABLEAU/CHARTS
print(f"\n💾 Saving dataset optimized for Tableau and line charts...")

# Create the final clean dataset
final_columns = [
    'record_id',        # Unique identifier
    'timestamp',        # Properly formatted datetime
    'device_id',        # Package identifier  
    'data_type',        # Sensor type (temperature/humidity/shock)
    'data_value',       # Original string value
    'numeric_value',    # Cleaned numeric value
    'location',         # GPS coordinates
    'closest_city',     # City name
    'status',           # Package status
    'origin',           # Origin city
    'destination'       # Destination city
]

df_final = df[final_columns].copy()

# Save main dataset
try:
    df_final.to_csv("cleaned_iot_data.csv", index=False)
    print("✅ Main dataset saved: cleaned_iot_data.csv")
except Exception as e:
    print(f"❌ Error saving main dataset: {e}")

# Save sensor-specific datasets for individual analysis
try:
    for sensor_type in ["temperature", "humidity", "shock"]:
        sensor_df = df_final[df_final['data_type'] == sensor_type].copy()
        filename = f"cleaned_{sensor_type}_data.csv"
        sensor_df.to_csv(filename, index=False)
        print(f"✅ {sensor_type.title()} dataset saved: {filename}")
except Exception as e:
    print(f"❌ Error saving sensor datasets: {e}")

# Save summary statistics
try:
    summary_data = []
    for sensor_type, stats in sensor_stats.items():
        summary_data.append({
            'sensor_type': sensor_type,
            'total_records': stats['count'],
            'min_value': stats['min'],
            'max_value': stats['max'],
            'avg_value': stats['mean'],
            'std_deviation': stats['std']
        })
    
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv("iot_data_summary.csv", index=False)
    print("✅ Summary statistics saved: iot_data_summary.csv")
except Exception as e:
    print(f"❌ Error saving summary: {e}")

# Step 7: TABLEAU PREPARATION NOTES
print(f"\n📋 TABLEAU PREPARATION COMPLETE!")
print("=" * 70)
print("📁 FILES CREATED:")
print("   cleaned_iot_data.csv - Main dataset for Tableau dashboard")
print("   cleaned_temperature_data.csv - Temperature sensor only")  
print("   cleaned_humidity_data.csv - Humidity sensor only")
print("   cleaned_shock_data.csv - Shock sensor only")
print("   iot_data_summary.csv - Statistical summary")

print(f"\n🎉 SUCCESS! Dataset ready for visualization!")
print(f"📊 Total: {len(df_final)} records from {df_final['device_id'].nunique()} packages")
print(f"🕐 Original timeline preserved: {df_final['timestamp'].min()} to {df_final['timestamp'].max()}")
print(f"✅ ALL VALUES POSITIVE - No negative readings in dataset")
print("🚀 Ready for Tableau dashboard and line chart creation!")

📥 Starting Week 6: Data Retrieval and Processing (CORRECTED VERSION)
🎯 Output: Clean dataset ready for Tableau dashboards and line charts
🔗 Connecting to blockchain...
✅ Connected to Ganache successfully!
✅ Connected to Smart Contract

📁 Looking for logistics data file...
📋 CSV files found: ['cleaned_humidity_data.csv', 'cleaned_iot_data.csv', 'cleaned_shock_data.csv', 'cleaned_temperature_data.csv', 'iot_data_summary.csv', 'Logistics-Data.csv', 'W6-Cleaned_Iot_Data.csv']
✅ Found and loaded: Logistics-Data.csv
📊 Data shape: (100, 10)
✅ All required columns found!

📊 Checking blockchain data...
📦 Total packages: 100
📈 Blockchain records found: 300

📥 Retrieving 300 records from blockchain...
🕐 Using ORIGINAL CSV timestamps (not blockchain timestamps)
📅 Timeline will match your original logistics data
📊 Original data timeline: 2025-04-29 12:08:30.336512 to 2025-05-06 10:09:30.336512
🔍 Testing first device: PKG85046
📊 Test device has 3 records
🧪 Test record structure: ['temperature', '26.